In [92]:
%load_ext autoreload
%autoreload 2

import oddesa
import importlib
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
oddesa.equation("x' = x - k*x|tau + x|1 + x'|1", 
                k=2, tau = np.linspace(1,2,1001))


x - k*x|tau + x|1 + x'|1
{'x': "x - k*x|tau + x|1 + x'|1"}
['x']
{'tau', '1'}
{('x', '1'), ('x', 'tau'), ("x'", '1')}


equation(x' = x - k*x|tau + x|1 + x'|1) where (k = 2; tau = [1.    1.001 1.002 ... 1.998 1.999 2.   ])

In [91]:
oddesa.equation("x'' = x - k * x|tau + x|1.5 + x''|tau", 
                k=2, tau=1)

x - k * x|tau + x|1.5 + x''|tau
x'
{"x'": "x - k * x|tau + x|1.5 + x''|tau", 'x': "x'"}
["x'", 'x']
{'1.5', 'tau'}
{("x''", 'tau'), ('x', 'tau'), ('x', '1.5')}


equation(x'' = x - k * x|tau + x|1.5 + x''|tau; x' = x') where (k = 2; tau = 1)

In [75]:
oddesa.cpp_errors("int x˛ = 0; int main() {return 0;}")

''

In [161]:
lorenz_eq = oddesa.equation("""
    x'''' = sigma*(y - x''|1.02); 
    y' = rho*x - y'|1 - x*z;
    z' = -beta*z + x*z;
""", sigma=1, rho=28, beta=3/8)

lorenz_eq.print_cpp_parameters()


        #include <iostream>
        using namespace std;
        int main() {
            cout  
                << "sigma" << " = " << 1 << "; "  
                << "rho" << " = " << 28 << "; "  
                << "beta" << " = " << 0.375 << "; " << endl;
        }
        
sigma = 1; rho = 28; beta = 0.375; 



In [138]:
f"{type(2)}"

"<class 'int'>"